In [1]:
import os
import pandas as pd
import pickle
import json


from src.dataset.dataset_info import datasets
from src.dataset.clean_dataset import clean_dataset
from src.dataset.create_class_num_col import one_dataset_class_num_col
from src.dataset.features_analysis import feature_analysis_pipeline
from src.dataset.undersample_classes import undersample_classes
from src.graph.graph_measures import calculate_graph_measures
from local_variables import datasets_main_path


In [2]:
with_sort_timestamp = False
with_undersample_classes = False

# name = "cic_ids_2017_5_percent"
# original_path = "./testing_dfs/cic_ids_2017_5_percent.parquet"
# file_type = "parquet"

# name = "cic_ton_iot_5_percent"
# original_path = "./testing_dfs/cic_ton_iot_5_percent.parquet"
# file_type = "parquet"

# name = "cic_ton_iot"
# original_path = datasets_main_path + "cic_ton_iot.parquet"
# file_type = "parquet"

# name = "cic_ids_2017"
# original_path = datasets_main_path + "cic_ids_2017.parquet"
# file_type = "parquet"

name = "cic_ton_iot_modified"
original_path = datasets_main_path + "CIC-ToN-IoT-Modified.pkl"
# original_path = datasets_main_path + "CCD-INID-Modified.pkl"
file_type = "pkl"




# name = "nf_ton_iotv2_modified"
# name = "ccd_inid_modified"
# name = "nf_uq_nids_modified"


# name = "nf_bot_iot"

# name = "edge_iiot"
# name = "nf_cse_cic_ids2018"
# name = "nf_bot_iotv2"
# name = "nf_uq_nids"
# name = "x_iiot"

folder_path = os.path.join("datasets", name)
output_path = os.path.join(folder_path, name + ".parquet")

if not os.path.isdir(folder_path):
    os.mkdir(folder_path)

print(f"==>> original_path: {original_path}")
print(f"==>> folder_path: {folder_path}")
print(f"==>> output_path: {output_path}")


==>> original_path: c:\Users\Administrateur\Desktop\datasets\CIC-ToN-IoT-Modified.pkl
==>> folder_path: datasets\cic_ton_iot_modified
==>> output_path: datasets\cic_ton_iot_modified\cic_ton_iot_modified.parquet


# Preparing Datasets

### Reading and Cleaning

In [3]:
if file_type == "parquet":
    df = pd.read_parquet(original_path)
if file_type == "csv":
    df = pd.read_csv(original_path)
if file_type == "pkl":
    df = pd.read_pickle(original_path, compression="zip")

In [4]:
df.head()

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,...,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,Attack,datetime
0,177.30.87.144-192.168.1.1-0-0-0,177.30.87.144,0,192.168.1.1,0,0,25/04/2019 05:18:52 pm,47814343,5,0,...,0.000000e+00,1038036.0,1038036.0,5.187256e+14,8.984590e+14,1.556177e+15,1.657324e+07,0,Benign,2019-04-25 17:18:52
1,167.49.176.28-50.165.192.168-0-0-0,167.49.176.28,0,50.165.192.168,0,0,25/04/2019 05:18:49 pm,2033142,2,0,...,0.000000e+00,0.0,0.0,1.556177e+15,0.000000e+00,1.556177e+15,1.556177e+15,0,Benign,2019-04-25 17:18:49
2,230.158.52.59-177.21.192.168-0-0-0,230.158.52.59,0,177.21.192.168,0,0,25/04/2019 05:18:37 pm,82877133,14,0,...,1.711593e+06,3942470.0,226402.0,1.729085e+14,5.187256e+14,1.556177e+15,6.036493e+06,0,Benign,2019-04-25 17:18:37
3,183.68.192.168-1.1.192.168-0-0-0,183.68.192.168,0,1.1.192.168,0,0,25/04/2019 05:18:42 pm,24359,2,0,...,0.000000e+00,0.0,0.0,1.556177e+15,0.000000e+00,1.556177e+15,1.556177e+15,0,Benign,2019-04-25 17:18:42
4,183.41.192.168-1.1.192.168-0-0-0,183.41.192.168,0,1.1.192.168,0,0,25/04/2019 05:18:42 pm,10239351,3,0,...,0.000000e+00,4053975.0,4053975.0,7.780884e+14,1.100383e+15,1.556177e+15,6.185376e+06,0,Benign,2019-04-25 17:18:42


In [5]:
timestamp_format = "mixed"
# timestamp_format="%d/%m/%Y %I:%M:%S %p"

In [6]:
df.columns

Index(['Flow ID', 'Src IP', 'Src Port', 'Dst IP', 'Dst Port', 'Protocol',
       'Timestamp', 'Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts',
       'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max',
       'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std',
       'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean',
       'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean',
       'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot',
       'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min',
       'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max',
       'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags',
       'Bwd URG Flags', 'Fwd Header Len', 'Bwd Header Len', 'Fwd Pkts/s',
       'Bwd Pkts/s', 'Pkt Len Min', 'Pkt Len Max', 'Pkt Len Mean',
       'Pkt Len Std', 'Pkt Len Var', 'FIN Flag Cnt', 'SYN Flag Cnt',
       'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'URG Flag Cnt',
       'CWE Flag Count', 'ECE 

In [15]:
list(df.dtypes)

[dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('int64'),
 dtype('O'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype

In [9]:
src_ip_col="Src IP"
src_port_col="Src Port"
dst_ip_col="Dst IP"
dst_port_col="Dst Port"
flow_id_col="Flow ID"
timestamp_col="Timestamp"
label_col="Label"
class_col="Attack"
class_num_col="Class"

In [10]:
drop_columns=[flow_id_col, src_ip_col, dst_ip_col, timestamp_col, src_port_col, dst_port_col, class_col]

In [11]:
df = clean_dataset(df, timestamp_col, flow_id_col)

In [12]:
def to_categorical_codes(column):
        return pd.Categorical(column).codes
    
if name=='x_iiot':
    columns_to_convert = [
          'Protocol', 'Service', 'is_syn_only', 'Is_SYN_ACK', 
           'is_pure_ack', 'is_with_payload', 'FIN or RST', 
        'Bad_checksum', 'is_SYN_with_RST', 'anomaly_alert']


    for col in columns_to_convert:
        if col in df.columns:
            df[col] = to_categorical_codes(df[col])
        else:
            print(f"Warning: Column '{col}' not found in the DataFrame")
            
    df['class3'] = (df['class3'] == 'Attack').astype(int)
    df = df.applymap(lambda x: 0 if x == '-' else x)
    df = df.applymap(lambda x: 0 if x == '?' else x)
    df = df.applymap(lambda x: 0 if x == '#DIV/0!' else x)    
    df = df.applymap(lambda x: 0 if x == 'excel' else x)
    df = df.applymap(lambda x: 0 if x == 'aza' else x)
    df = df.applymap(lambda x: 0 if x == ' ' else x)
    
if name=='edge_iiot':
    columns_to_convert = ['mqtt.protoname','mqtt.topic','mqtt.conack.flags','mqtt.msg','http.request.method','dns.qry.name.len','arp.src.proto_ipv4',
                          'http.request.full_uri','http.file_data','http.request.version','arp.dst.proto_ipv4','http.request.uri.query','tcp.srcport','http.referer']

    for col in columns_to_convert:
        if col in df.columns:
            df[col] = to_categorical_codes(df[col])
        else:
            print(f"Warning: Column '{col}' not found in the DataFrame")
    
if name=='ccd_inid_modified':    
    columns_to_convert = ['splt_direction','splt_ps','splt_piat_ms','application_name','application_category_name','requested_server_name','client_fingerprint']

    for col in columns_to_convert:
        if col in df.columns:
            df[col] = to_categorical_codes(df[col])
        else:
            print(f"Warning: Column '{col}' not found in the DataFrame")
    df['traffic_type'] = (df['traffic_type'] == 'attack').astype(int)

if name=='cic_ton_iot_modified':
    df.drop(['datetime'], axis=1)
    

In [13]:
df[src_ip_col] = df[src_ip_col].apply(str)
df[src_port_col] = df[src_port_col].apply(str)
df[dst_ip_col] = df[dst_ip_col].apply(str)
df[dst_port_col] = df[dst_port_col].apply(str)

In [14]:
_,var_dropped, corr_dropped = feature_analysis_pipeline(df=df, drop_columns=drop_columns,label_col=label_col)
var_dropped, corr_dropped

DTypePromotionError: The DType <class 'numpy.dtypes.DateTime64DType'> could not be promoted by <class 'numpy.dtypes.Float64DType'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single array unless the dtype is `object`. The full list of DTypes is: (<class 'numpy.dtypes.Int64DType'>, <class 'numpy.dtypes.Int64DType'>, <class 'numpy.dtypes.Int64DType'>, <class 'numpy.dtypes.Int64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <

### Attacks Types

In [ ]:
df[class_col].unique()

In [ ]:
# df[class_col] = df[class_col].replace({"BENIGN": "Benign",
#                                        "DDoS": "ddos",
#                                        "Web Attack � Brute Force": "bruteforce",
#                                        "Web Attack � XSS": "xss"})

In [ ]:
classes = df[class_col].unique()

### Sorting (optional)

In [ ]:
if with_sort_timestamp:
    df[timestamp_col] = pd.to_datetime(df[timestamp_col].str.strip(), format=timestamp_format)
    df.sort_values(timestamp_col, inplace= True)

### Encoding Attacks into integers

In [ ]:
df, labels_names = one_dataset_class_num_col(df, class_num_col, class_col)

### Undersampling classes (optional)

In [ ]:
if with_undersample_classes:
    # Group by the class column and get the count of records in each class
    class_counts = df.groupby(class_col).size()

    # Sort the counts in descending order
    class_counts_sorted = class_counts.sort_values(ascending=False)
    print(f"==>> class_counts_sorted: {class_counts_sorted}")

In [ ]:
if with_undersample_classes:
    # Get the classes with the highest number of records (you can choose how many to undersample)
    classes_to_undersample = class_counts_sorted.index[:2]

    # Undersample the classes with the highest number of records
    dfs = []
    for class_label in class_counts_sorted.index:
        print(f"==>> class_label: {class_label}")
        if class_label in classes_to_undersample:
            class_df = df[df[class_col] == class_label]
            undersampled_df = class_df.sample(frac=0.5)  # Specify the fraction of samples to keep
            dfs.append(undersampled_df)
        else:
            dfs.append(df[df[class_col] == class_label])

    df = []
    # Optional: shuffle the undersampled DataFrame
    df = pd.concat(dfs).sample(frac=1).reset_index(drop=True)


In [ ]:
if with_undersample_classes:
    # Group by the class column and get the count of records in each class
    class_counts = df.groupby(class_col).size()

    # Sort the counts in descending order
    class_counts_sorted = class_counts.sort_values(ascending=False)
    print(f"==>> class_counts_sorted: {class_counts_sorted}")

### saving labels encodings and datasets properties

In [ ]:
with open(folder_path + '/labels_names.pkl', 'wb') as f:
    pickle.dump([labels_names, classes], f)

In [ ]:
total_count = len(df)

properties = {
    "name": name,
    "length": total_count,
}

num_benign = len(df[df['Label'] == 0])
num_attack = len(df[df['Label'] == 1])

properties["num_benign"] = num_benign
properties["percentage_of_benign_records"] = ((num_benign * 100)/total_count)

properties["num_attack"] = num_attack
properties["percentage_of_attack_records"] = ((num_attack * 100)/total_count)

properties["attacks"] = list(df["Attack"].unique()) 

with open(folder_path + '/df_properties.json', 'w') as f:
    json.dump(properties, f)
    
properties


In [ ]:
# graphs_properties_path = folder_path + '/graphs_properties.json'
# G = nx.from_pandas_edgelist(
#     df,
#     source=src_ip_col,
#     target=dst_ip_col,
#     create_using=nx.DiGraph()
# )
# calculate_graph_measures(G, graphs_properties_path)

In [ ]:
df.to_parquet(output_path)